In [11]:
MINGPT_PATH = 'genai\\minGPT'
DATA_SRC_PATH = 'genai\\minGPT_history_books\\data_source'

import sys

sys.path.insert(0, MINGPT_PATH)
sys.path.insert(0, DATA_SRC_PATH)

import numpy as np 
import pandas as pd # 
import os

import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

from mingpt.model import GPT
from mingpt.trainer import Trainer
from mingpt.utils import set_seed, setup_logging, CfgNode as CN
import re
import os

set_seed(42)

In [12]:
def open_n_preprocess_text(path):
    input_text = open(path, encoding="utf8").read()
    # Remove substrings with numbers between spaces
    processed_text = re.sub(r'\s\d+\s', '\n', input_text)

    # Reduce consecutive newlines to a maximum of two
    processed_text = re.sub(r'\n{3,}', '\n\n', processed_text)

    # Remove leading and trailing whitespaces
    processed_text = processed_text.strip()

    return processed_text

def count_words(input_string):
    words = input_string.split()
    print(f"number of words: {len(words)}")

In [16]:
# List all files in the directory
files = os.listdir(DATA_SRC_PATH)
files_read = []
for file_name in files:
    if file_name.endswith(".txt"):
        file_path = os.path.join(DATA_SRC_PATH, file_name)
        text1 = open_n_preprocess_text(file_path)
        count_words(text1)
        files_read.append(text1)
data = '\n\n '.join(files_read)
count_words(data)

number of words: 53270
number of words: 117283
number of words: 33401
number of words: 128890
number of words: 75539
number of words: 49475
number of words: 79785
number of words: 4628
number of words: 18976
number of words: 60574
number of words: 621821


In [17]:
def get_config():

    C = CN()

    # system
    C.system = CN()
    C.system.seed = 3407
    C.system.work_dir = './out/chargpt'

    # data
    C.data = CharDataset.get_default_config()

    # model
    C.model = GPT.get_default_config()
    C.model.model_type = 'gpt-mini'

    # trainer
    C.trainer = Trainer.get_default_config()
    C.trainer.learning_rate = 4e-4 # the model we're using is so small that we can go a bit faster
    C.trainer.num_workers = 0 # error when in localhost

    return C

In [18]:
class CharDataset(Dataset):
    """
    Emits batches of characters
    """

    @staticmethod
    def get_default_config():
        C = CN()
        C.block_size = 128
        return C

    def __init__(self, config, data):
        self.config = config

        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))

        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.vocab_size = vocab_size
        self.data = data

    def get_vocab_size(self):
        return self.vocab_size

    def get_block_size(self):
        return self.config.block_size

    def __len__(self):
        return len(self.data) - self.config.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.config.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        # return as tensors
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [19]:
# get default config and overrides from the command line, if any
config = get_config()
print(config)
setup_logging(config)
set_seed(config.system.seed)

# construct the training dataset
train_dataset = CharDataset(config.data, data)

system:
    seed: 3407
    work_dir: ./out/chargpt
data:
    block_size: 128
model:
    model_type: gpt-mini
    n_layer: None
    n_head: None
    n_embd: None
    vocab_size: None
    block_size: None
    embd_pdrop: 0.1
    resid_pdrop: 0.1
    attn_pdrop: 0.1
trainer:
    device: auto
    num_workers: 0
    max_iters: None
    batch_size: 64
    learning_rate: 0.0004
    betas: (0.9, 0.95)
    weight_decay: 0.1
    grad_norm_clip: 1.0

data has 4548434 characters, 115 unique.


In [20]:
# construct the model
config.model.vocab_size = train_dataset.get_vocab_size()
config.model.block_size = train_dataset.get_block_size()
model = GPT(config.model)

number of parameters: 2.72M


In [21]:
# construct the trainer object
trainer = Trainer(config.trainer, model, train_dataset)

running on device cpu


In [ ]:
# iteration callback
def batch_end_callback(trainer):

    if trainer.iter_num % 10 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")

    if trainer.iter_num % 500 == 0:
        # evaluate both the train and test score
        model.eval()
        with torch.no_grad():
            # sample from the model...
            context = "Apakah Sejarah malaysia?"
            x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
            y = model.generate(x, 500, temperature=1.0, do_sample=True, top_k=10)[0]
            completion = ''.join([train_dataset.itos[int(i)] for i in y])
            print(completion)
        # save the latest model
        print("saving model")
        ckpt_path = os.path.join(config.system.work_dir, "model.pt")
        torch.save(model.state_dict(), ckpt_path)
        # revert model to training mode
        model.train()

trainer.set_callback('on_batch_end', batch_end_callback)

# run the optimization
trainer.run()